In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 全結合層の定義

In [2]:
# シード値固定
torch.manual_seed(1)

# 全結合層の定義
fc = nn.Linear(3, 2) # 3次元から2次元へ

In [3]:
# 重み(W)：ランダムに格納
fc.weight

Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462]], requires_grad=True)

In [4]:
# バイアス(b)：ランダムに格納
fc.bias

Parameter containing:
tensor([-0.1188,  0.2937], requires_grad=True)

### 線形変換

In [5]:
# PyTorchで使用するTensor型に変換(各値はfloat型に)
x = torch.tensor([[1, 2, 3]], dtype=torch.float32)

In [6]:
# 線形変換を実行
u = fc(x)
u

tensor([[-0.6667,  0.5164]], grad_fn=<AddmmBackward0>)

### 非線形変換(ReLU関数による)

In [7]:
# ReLU関数による変換
z = F.relu(u)
z

tensor([[0.0000, 0.5164]], grad_fn=<ReluBackward0>)

### 例題

In [8]:
torch.manual_seed(1)
fc = nn.Linear(3, 2)
fc_2 = nn.Linear(2, 1)

x = torch.tensor([[1, 2, 3]], dtype=torch.float32)

z = F.relu(fc(x))
y = fc_2(z)

y

tensor([[0.1514]], grad_fn=<AddmmBackward0>)

### 目的関数(平均二乗誤差による)

In [9]:
# 目標値の設定
t = torch.tensor([[1]], dtype=torch.float32)
t

tensor([[1.]])

In [10]:
# MSEの算出
loss = F.mse_loss(t, y)
loss

tensor(0.7201, grad_fn=<MseLossBackward0>)

## 実装（基礎編）

### データセットの準備

In [11]:
from sklearn.datasets import load_iris
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [12]:
# 特徴量と目標値の定義
x, t = load_iris(return_X_y=True)

In [13]:
# torch型に変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [14]:
# まとめて一つのオブジェクトdatasetに変換
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [15]:
# データセットを学習用,検証用,テスト用に分割(train_test_splitと同じ？)
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

### ミニバッチ学習

In [16]:
batch_size = 10
# shuffleは学習データのみTrueに(目的関数を変化させることで,局所解に囚われないように)
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

### モデルの定義

In [17]:
# インスタンスnetの作成
class Net(nn.Module):
    # 使用するオブジェクトの定義
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    
    # 順伝播
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

In [18]:
torch.manual_seed(0)
# インスタンス化
net = Net()

In [19]:
# モデルの確認
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

### 目的関数の選択(クロスエントロピーによる)

In [20]:
# 目的関数の定義
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Optional[torch.Tensor] = None, size_average: Optional[bool] = None, ignore_index: int = -100, reduce: Optional[bool] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

### 最適化手法の選択

In [21]:
# パラメータの取得
for parameter in iter(net.parameters()):
    print(parameter)

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)


In [22]:
# 最適化手法の選択(今回はSGD：確率的勾配降下法)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) # lr:学習係数
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

### 学習

In [23]:
# バッチサイズ分のサンプルを抽出
batch = next(iter(train_loader)) # iter: 1イテレーション分取り出せる, next: 順に
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [24]:
# 入力値と目標値に分割
x, t = batch

In [25]:
# 全結合層fc1のパラメータ(重み)の値
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [26]:
# 全結合層fc1のバイアスの値
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)

In [27]:
# 予測値の算出
y = net(x) # net.forward(x)と書いても同じ
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [28]:
# 目的関数の算出
loss = criterion(y, t)
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [29]:
# 勾配の算出
loss.backward()

In [30]:
# fc1の重みに関する勾配が求まった
net.fc1.weight.grad

tensor([[-0.2311, -0.1731, -0.0627, -0.0139],
        [ 0.7327,  0.3358,  0.6025,  0.2229],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [31]:
# fc1のバイアスに関する勾配が求まった
net.fc1.bias.grad

tensor([-0.0461,  0.1060,  0.0000,  0.0000])

In [32]:
# 勾配情報を用いたパラメータの更新
optimizer.step()

### データのデバイスへの転送

In [33]:
# GPUが使用可能かの確認
torch.cuda.is_available()

False

In [34]:
# GPUが使用可能ならcuda、そうでなければcpuを選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [35]:
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [36]:
# デバイスへの入力変数の転送
x = x.to(device)
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [37]:
# デバイスへの目標値の転送
t = t.to(device)
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

### 勾配の初期化

In [38]:
# パラメータの勾配を求める前に勾配情報の初期化が必要
optimizer.zero_grad()

## モデルの学習(まとめ)

In [39]:
# エポック数は1
max_epoch = 1

In [40]:
torch.manual_seed(0)
# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

In [41]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [42]:
# 学習
for epoch in range(max_epoch):
    
    for batch in train_loader:
        
        # バッチサイズ分のサンプルを抽出
        x, t = batch
        
        # 学習時に使用するデバイスへデータの転送
        x = x.to(device)
        t = t.to(device)
        
        # パラメータの勾配を初期化
        optimizer.zero_grad()
        
        # 予測値の算出
        y = net(x)
        
        # 目標値と予測値から目的関数の値を算出
        loss = criterion(y, t)
        
        # 目的関数の値を表示して確認
        # item(): tensot.Tensor => float
        print('loss: ', loss.item())
        
        # 各パラメータの勾配を算出
        loss.backward()
        
        # 勾配の情報を用いたパラメータの更新
        optimizer.step()

loss:  1.0881630182266235
loss:  1.0393922328948975
loss:  1.0028115510940552
loss:  1.0250868797302246
loss:  1.0088638067245483
loss:  0.9351975321769714
loss:  0.8939588665962219
loss:  0.9765418171882629
loss:  0.9651517868041992


In [43]:
# 「行ごと/列ごと」の最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)# dim=1 で「行ごと」に
y_label

tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 0])

In [44]:
# 目的変数
t

tensor([1, 1, 0, 2, 1, 2, 0, 2, 1, 1])

In [45]:
# 正解率の算出
acc = torch.sum(y_label == t) * 1.0 / len(t) # 切り捨て防止のためにfloatへ変換
acc

tensor(0.4000)

In [46]:
# 学習(改)
for epoch in range(max_epoch):

    for batch in train_loader:

        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)

        # New：正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc  = torch.sum(y_label == t) * 1.0 / len(t)
        print('accuracy:', acc)

        loss.backward()
        optimizer.step()

accuracy: tensor(0.7000)
accuracy: tensor(0.2000)
accuracy: tensor(0.8000)
accuracy: tensor(0.5000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.4000)
accuracy: tensor(0.8000)
accuracy: tensor(0.2000)


### 評価用

In [47]:
# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc

In [48]:
calc_acc(val_loader)

Accuracy: 70.0%


tensor(0.7000)

In [49]:
calc_acc(test_loader)

Accuracy: 70.0%


tensor(0.7000)